In [8]:
# Загрузка данных, первичная подготовка
import pandas as pd

df = pd.read_csv('D:\\ALL\\Step_Doc\\_VLesson\\_DataAnalytic\\finuniver\\_Itog\\Выполнение\\кейс 31\\4.1.0_ETL Exel\\Product.csv', sep=';')

#Подготовка данных
df.info()
df['StandardCost']=df['StandardCost'].astype(float)
df['StartDate'] = pd.to_datetime(df['StartDate'], format='%d.%m.%Y')
columns_to_convert = ['SubCategory', 'Category']
df[columns_to_convert] = df[columns_to_convert].astype('category')
columns_to_convert = ['ProductName','ProductLine', 'ModelName', 'Color', 'ProductDescription']
df[columns_to_convert] = df[columns_to_convert].astype('string')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ProductKey          606 non-null    int64  
 1   ProductName         606 non-null    object 
 2   SubCategory         397 non-null    object 
 3   Category            397 non-null    object 
 4   StandardCost        395 non-null    float64
 5   Color               352 non-null    object 
 6   ListPrice           395 non-null    float64
 7   DaysToManufacture   606 non-null    int64  
 8   ProductLine         380 non-null    object 
 9   ModelName           397 non-null    object 
 10  Photo               606 non-null    object 
 11  ProductDescription  396 non-null    object 
 12  StartDate           606 non-null    object 
dtypes: float64(2), int64(2), object(9)
memory usage: 61.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 13 co

In [9]:
'''
Моделирование и предсказание категориальных признаков
RandomForest, nltk (для работы с естественным языком)
'''

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
import nltk

# Проверка наличия данных токенизатора NLTK
nltk.download('punkt')

# Создание копии DataFrame для работы с известными категориями
df_known = df.dropna(subset=['Category', 'SubCategory']).copy()

# Токенизация названий продуктов с использованием .loc для явного назначения
df_known.loc[:, 'tokens'] = df_known['ProductName'].astype(str).apply(nltk.word_tokenize)

# Обучение модели Word2Vec
word2vec_model = Word2Vec(sentences=df_known['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Функция для получения усредненного вектора для каждого названия продукта
def get_vector(text):
    if pd.isna(text):  # Проверка на наличие NaN
        return np.zeros(word2vec_model.vector_size)  # Возвращаем вектор нулей, если текст отсутствует
    
    words = nltk.word_tokenize(text)
    vector = np.zeros(word2vec_model.vector_size)
    count = 0
    for word in words:
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    return vector / count if count > 0 else vector

# Функция для вывода  категориальных метрик 
def evaluate_model(model, X_test, y_test, label):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)
    
    print(f"Оценка модели для {label}:")
    print(f"Точность (Accuracy): {accuracy:.4f}")
    print(f"Точность (Precision): {precision:.4f}")
    print(f"Полнота (Recall): {recall:.4f}")
    print(f"F1-оценка: {f1:.4f}")
    print("\n")

# --------- Шаг 1: Обучение и предсказание для Category ---------

# Применение функции для получения векторов
X_category = np.array([get_vector(text) for text in df_known['ProductName']])

# Проверка на наличие NaN в X_category
X_category = np.nan_to_num(X_category)

# Целевая переменная для Category
y_category = df_known['Category']

# Удаление строк с NaN в целевой переменной
valid_idx_category = y_category.notna()
X_category = X_category[valid_idx_category]
y_category = y_category[valid_idx_category]

# Обучение модели Random Forest для Category
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_category, y_category, test_size=0.2, random_state=42)
model_category = RandomForestClassifier(n_estimators=100, random_state=42)
model_category.fit(X_train_cat, y_train_cat)

# Оценка модели для Category
evaluate_model(model_category, X_test_cat, y_test_cat, "Category")

# Применение модели для заполнения Category
df_missing_category = df[df['Category'].isnull()].copy()
if not df_missing_category.empty:
    X_missing_category = np.array([get_vector(text) for text in df_missing_category['ProductName']])
    X_missing_category = np.nan_to_num(X_missing_category)  # Замена NaN на 0
    df_missing_category['Category'] = model_category.predict(X_missing_category)
    
    # Обновление исходного DataFrame
    df.update(df_missing_category[['Category']])

# --------- Шаг 2: Обучение и предсказание для SubCategory ---------

# Применение функции для получения векторов
X_subcategory = np.array([get_vector(text) for text in df_known['ProductName']])

# Проверка на наличие NaN в X_subcategory
X_subcategory = np.nan_to_num(X_subcategory)

# Целевая переменная для SubCategory
y_subcategory = df_known['SubCategory']

# Удаление строк с NaN в целевой переменной
valid_idx_subcategory = y_subcategory.notna()
X_subcategory = X_subcategory[valid_idx_subcategory]
y_subcategory = y_subcategory[valid_idx_subcategory]

# Обучение модели Random Forest для SubCategory
X_train_subcat, X_test_subcat, y_train_subcat, y_test_subcat = train_test_split(X_subcategory, y_subcategory, test_size=0.2, random_state=42)
model_subcategory = RandomForestClassifier(n_estimators=100, random_state=42)
model_subcategory.fit(X_train_subcat, y_train_subcat)

# Оценка модели для SubCategory
evaluate_model(model_subcategory, X_test_subcat, y_test_subcat, "SubCategory")

# Применение модели для заполнения SubCategory
df_missing_subcategory = df[df['SubCategory'].isnull()].copy()
if not df_missing_subcategory.empty:
    X_missing_subcategory = np.array([get_vector(text) for text in df_missing_subcategory['ProductName']])
    X_missing_subcategory = np.nan_to_num(X_missing_subcategory)  # Замена NaN на 0
    df_missing_subcategory['SubCategory'] = model_subcategory.predict(X_missing_subcategory)
    
    # Обновление исходного DataFrame
    df.update(df_missing_subcategory[['SubCategory']])


df.head(100)


[nltk_data] Downloading package punkt to C:\Users\Render-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Оценка модели для Category:
Точность (Accuracy): 0.9625
Точность (Precision): 0.9646
Полнота (Recall): 0.9625
F1-оценка: 0.9601


Оценка модели для SubCategory:
Точность (Accuracy): 0.8250
Точность (Precision): 0.8943
Полнота (Recall): 0.8250
F1-оценка: 0.7875




,ProductKey,ProductName,SubCategory,Category,StandardCost,Color,ListPrice,DaysToManufacture,ProductLine,ModelName,Photo,ProductDescription,StartDate
0,1,Adjustable Race,Tires and Tubes,Accessories,NaN,<NA>,NaN,0,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
1,2,Bearing Ball,Tires and Tubes,Accessories,NaN,<NA>,NaN,0,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
2,3,BB Ball Bearing,Tires and Tubes,Accessories,NaN,<NA>,NaN,1,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
3,4,Headset Ball Bearings,Road Bikes,Components,NaN,<NA>,NaN,0,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
4,5,Blade,Tires and Tubes,Accessories,NaN,<NA>,NaN,1,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Internal Lock Washer 6,Locks,Accessories,NaN,<NA>,NaN,0,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
96,97,Internal Lock Washer 10,Locks,Accessories,NaN,<NA>,NaN,0,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
97,98,Internal Lock Washer 1,Locks,Accessories,NaN,<NA>,NaN,0,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01
98,99,Internal Lock Washer 8,Locks,Accessories,NaN,<NA>,NaN,0,<NA>,<NA>,http://www.avising.com/me/LearnPBI/DataSources...,<NA>,1998-06-01


In [10]:
#Заполнение неустановленных категориальных значений значением по умолчанию
columns_to_fill = ['ProductLine', 'ModelName', 'Color', 'ProductDescription']
df[columns_to_fill]= df[columns_to_fill].astype('string')
df[columns_to_fill] = df[columns_to_fill].fillna('NotSet')
df[columns_to_fill] = df[columns_to_fill].astype('category')

df.head(100)

,ProductKey,ProductName,SubCategory,Category,StandardCost,Color,ListPrice,DaysToManufacture,ProductLine,ModelName,Photo,ProductDescription,StartDate
0,1,Adjustable Race,Tires and Tubes,Accessories,NaN,NotSet,NaN,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
1,2,Bearing Ball,Tires and Tubes,Accessories,NaN,NotSet,NaN,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
2,3,BB Ball Bearing,Tires and Tubes,Accessories,NaN,NotSet,NaN,1,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
3,4,Headset Ball Bearings,Road Bikes,Components,NaN,NotSet,NaN,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
4,5,Blade,Tires and Tubes,Accessories,NaN,NotSet,NaN,1,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Internal Lock Washer 6,Locks,Accessories,NaN,NotSet,NaN,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
96,97,Internal Lock Washer 10,Locks,Accessories,NaN,NotSet,NaN,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
97,98,Internal Lock Washer 1,Locks,Accessories,NaN,NotSet,NaN,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
98,99,Internal Lock Washer 8,Locks,Accessories,NaN,NotSet,NaN,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01


In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ProductKey          606 non-null    int64         
 1   ProductName         606 non-null    string        
 2   SubCategory         606 non-null    object        
 3   Category            606 non-null    object        
 4   StandardCost        395 non-null    float64       
 5   Color               606 non-null    category      
 6   ListPrice           395 non-null    float64       
 7   DaysToManufacture   606 non-null    int64         
 8   ProductLine         606 non-null    category      
 9   ModelName           606 non-null    category      
 10  Photo               606 non-null    object        
 11  ProductDescription  606 non-null    category      
 12  StartDate           606 non-null    datetime64[ns]
dtypes: category(4), datetime64[ns](1), float64(2), int

In [12]:
'''
Моделирование и предсказание количественных признаков
RandomForestRegressor, pandas
'''
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Разделяем признаки и целевые переменные для столбца StandardCost
X = df.drop(columns=['StandardCost', 'ListPrice'])
y_standard = df['StandardCost']

# Выбираем категориальные и числовые признаки
categorical_features = X.select_dtypes(include=['category', 'object']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

# Преобразование категориальных данных и заполнение пропусков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))  # Заполняем пропуски средним
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Заполняем пропуски наиболее частым значением
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot кодирование
])

# Комбинируем обработчики
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Создаем модель с RandomForest
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Разделяем данные на обучающую и тестовую выборки для StandardCost
X_train, X_test, y_train, y_test = train_test_split(X[~y_standard.isna()], y_standard.dropna(), test_size=0.2, random_state=42)

# Обучаем модель для StandardCost
model.fit(X_train, y_train)

# Оценка на тестовых данных для StandardCost
y_pred = model.predict(X_test)

# Вычисление метрик для StandardCost
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Выводим метрики
print(f'Оценка модели для StandardCost:')
print(f'Коэффициент детерминации (R²): {r2:.4f}')
print(f'Среднеквадратичная ошибка (MSE): {mse:.4f}')
print(f'Средняя абсолютная ошибка (MAE): {mae:.4f}')

# Предсказание пропусков для StandardCost
X_missing = X[y_standard.isna()]
predicted_standard_cost = model.predict(X_missing)

# Заполняем пропуски для StandardCost
df.loc[df['StandardCost'].isna(), 'StandardCost'] = predicted_standard_cost

# Повторяем процесс для ListPrice
y_listprice = df['ListPrice']
X_train_lp, X_test_lp, y_train_lp, y_test_lp = train_test_split(X[~y_listprice.isna()], y_listprice.dropna(), test_size=0.2, random_state=42)

# Обучаем и заполняем для ListPrice
model.fit(X_train_lp, y_train_lp)

# Оценка точности модели для ListPrice
y_pred_lp = model.predict(X_test_lp)  # Добавляем предсказание для ListPrice

r2_lp = r2_score(y_test_lp, y_pred_lp)
mse_lp = mean_squared_error(y_test_lp, y_pred_lp)
mae_lp = mean_absolute_error(y_test_lp, y_pred_lp)

print(f'\nОценка модели для ListPrice:')
print(f'Коэффициент детерминации (R²): {r2_lp:.4f}')
print(f'Среднеквадратичная ошибка (MSE): {mse_lp:.4f}')
print(f'Средняя абсолютная ошибка (MAE): {mae_lp:.4f}')

# Предсказание для пропусков в ListPrice
X_missing_lp = X[y_listprice.isna()]
predicted_list_price = model.predict(X_missing_lp)

# Заполняем пропуски для ListPrice
df.loc[df['ListPrice'].isna(), 'ListPrice'] = predicted_list_price

df.head(10000)

Оценка модели для StandardCost:
Коэффициент детерминации (R²): 0.9883
Среднеквадратичная ошибка (MSE): 3171.4847
Средняя абсолютная ошибка (MAE): 34.9268

Оценка модели для ListPrice:
Коэффициент детерминации (R²): 0.9917
Среднеквадратичная ошибка (MSE): 6485.8408
Средняя абсолютная ошибка (MAE): 49.1701


,ProductKey,ProductName,SubCategory,Category,StandardCost,Color,ListPrice,DaysToManufacture,ProductLine,ModelName,Photo,ProductDescription,StartDate
0,1,Adjustable Race,Tires and Tubes,Accessories,24.6236,NotSet,43.614820,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
1,2,Bearing Ball,Tires and Tubes,Accessories,24.6236,NotSet,43.614820,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
2,3,BB Ball Bearing,Tires and Tubes,Accessories,28.7579,NotSet,64.575519,1,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
3,4,Headset Ball Bearings,Road Bikes,Components,30.6203,NotSet,61.548293,0,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
4,5,Blade,Tires and Tubes,Accessories,28.7579,NotSet,64.575519,1,NotSet,NotSet,http://www.avising.com/me/LearnPBI/DataSources...,NotSet,1998-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,602,ML Bottom Bracket,Bottom Brackets,Components,44.9500,NotSet,101.240000,1,NotSet,ML Bottom Bracket,http://www.avising.com/me/LearnPBI/DataSources...,Aluminum alloy cups; large diameter spindle.,2007-07-01
602,603,HL Bottom Bracket,Bottom Brackets,Components,53.9400,NotSet,121.490000,1,NotSet,HL Bottom Bracket,http://www.avising.com/me/LearnPBI/DataSources...,Aluminum alloy cups and a hollow axle.,2007-07-01
603,604,"Road-750 Black, 44",Road Bikes,Bikes,343.6500,Black,539.990000,4,Road,Road-750,http://www.avising.com/me/LearnPBI/DataSources...,Entry level adult bike; offers a comfortable r...,2007-07-01
604,605,"Road-750 Black, 48",Road Bikes,Bikes,343.6500,Black,539.990000,4,Road,Road-750,http://www.avising.com/me/LearnPBI/DataSources...,Entry level adult bike; offers a comfortable r...,2007-07-01


In [13]:
#Экспорт файла в csv - разделитьель ; т.к. есть сложный текст.
df.info()
#df.to_csv('Product_predicted.csv', index=False)
df.to_csv('1.1_Product_predicted.csv', index=False, sep=';')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ProductKey          606 non-null    int64         
 1   ProductName         606 non-null    string        
 2   SubCategory         606 non-null    object        
 3   Category            606 non-null    object        
 4   StandardCost        606 non-null    float64       
 5   Color               606 non-null    category      
 6   ListPrice           606 non-null    float64       
 7   DaysToManufacture   606 non-null    int64         
 8   ProductLine         606 non-null    category      
 9   ModelName           606 non-null    category      
 10  Photo               606 non-null    object        
 11  ProductDescription  606 non-null    category      
 12  StartDate           606 non-null    datetime64[ns]
dtypes: category(4), datetime64[ns](1), float64(2), int